In [72]:
import numpy as np
import pandas as pd
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler
import igraph as ig
import leidenalg
import matplotlib.pyplot as plt
import seaborn as sns
import os


def generate_synthetic_data(
    num_clusters, gene_mean_range, size_mean, size_std, genes=50
):
    np.random.seed(42)
    cluster_centers = np.random.uniform(
        gene_mean_range[0], gene_mean_range[1], num_clusters
    )
    cluster_sizes = np.abs(np.random.normal(size_mean, size_std, num_clusters)).astype(
        int
    )

    data = []
    for center, size in zip(cluster_centers, cluster_sizes):
        data.append(np.random.poisson(center, (size, genes)))

    data = np.vstack(data)
    gene_names = [f"Gene_{i}" for i in range(1, genes + 1)]
    cell_names = [f"Cell_{i}" for i in range(1, sum(cluster_sizes) + 1)]
    matrix = pd.DataFrame(data, index=cell_names, columns=gene_names)
    matrix.to_csv("sparse_gene_count_matrix.csv")
    print("Data creation complete and file saved.")
    return matrix, num_clusters


def load_data(filepath):
    return pd.read_csv(filepath, index_col=0)


def standardize_data(matrix):
    scaler = StandardScaler()
    return scaler.fit_transform(matrix)


def reduce_dimensionality(matrix_scaled, n_neighbors):
    reducer = umap.UMAP(n_neighbors=n_neighbors, random_state=42)
    embedding = reducer.fit_transform(matrix_scaled)
    return embedding, reducer


def create_igraph(reducer):
    sources, targets = reducer.graph_.tocoo().nonzero()
    weights = reducer.graph_[sources, targets].A1
    g = ig.Graph(list(zip(sources, targets)), directed=True)
    g.es["weight"] = [max(w, 0) for w in weights]
    return g


def leiden_clustering(graph, resolution):
    partition = leidenalg.find_partition(
        graph,
        leidenalg.RBConfigurationVertexPartition,
        resolution_parameter=resolution,
        weights="weight",
    )
    return np.array(partition.membership)


def plot_embedding(
    embedding, clusters, resolution, algorithm, real_num_clusters, plot_dir
):
    unique_clusters = np.unique(clusters)
    palette = sns.color_palette("pastel", len(unique_clusters))
    cluster_colors = {cluster: palette[i] for i, cluster in enumerate(unique_clusters)}
    colors = [cluster_colors[cluster] for cluster in clusters]

    plt.figure(figsize=(8, 8))
    plt.scatter(embedding[:, 0], embedding[:, 1], s=80, c=colors)
    plt.title(
        f"UMAP projection of the dataset\nResolution: {resolution}, Algorithm: {algorithm}, Real clusters: {real_num_clusters}"
    )
    plt.xlabel("UMAP 1")
    plt.ylabel("UMAP 2")

    plot_filename = f"{plot_dir}/UMAP_projection_res_{resolution}_clusters_{len(unique_clusters)}.pdf"
    plt.savefig(plot_filename, format="pdf")
    plt.close()
    print(f"Plot saved to {plot_filename}")


def main(num_clusters=3, gene_mean_range=(50, 200), size_mean=12, size_std=5):
    plot_dir = "clustering_plots"
    if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)

    # Step 1: Generate synthetic data
    matrix, real_num_clusters = generate_synthetic_data(
        num_clusters=num_clusters,
        gene_mean_range=gene_mean_range,
        size_mean=size_mean,
        size_std=size_std,
    )

    # Step 2: Load the data
    matrix = load_data("./sparse_gene_count_matrix.csv")

    # Step 3: Standardize the data
    matrix_scaled = standardize_data(matrix)

    # Step 4: Reduce dimensionality with UMAP
    n_neighbors = min(matrix_scaled.shape[0] - 1, 15)
    embedding, reducer = reduce_dimensionality(matrix_scaled, n_neighbors)

    # Step 5: Create igraph
    graph = create_igraph(reducer)

    # Step 6: Perform Leiden clustering with varying resolution
    for resolution in [0, 0.5, 1.0, 2.0, 10]:
        clusters = leiden_clustering(graph, resolution)
        print(f"Resolution: {resolution}, Number of clusters: {len(set(clusters))}")
        plot_embedding(
            embedding, clusters, resolution, "Leiden", real_num_clusters, plot_dir
        )


# Example parameters: 5 clusters, gene means from 50 to 200, sizes with mean 12 and std dev 5
main(num_clusters=120, gene_mean_range=(50, 200), size_mean=12, size_std=8)

Data creation complete and file saved.
Resolution: 0, Number of clusters: 1
Plot saved to clustering_plots/UMAP_projection_res_0_clusters_1.pdf
Resolution: 0.5, Number of clusters: 7
Plot saved to clustering_plots/UMAP_projection_res_0.5_clusters_7.pdf
Resolution: 1.0, Number of clusters: 8
Plot saved to clustering_plots/UMAP_projection_res_1.0_clusters_8.pdf
Resolution: 2.0, Number of clusters: 12
Plot saved to clustering_plots/UMAP_projection_res_2.0_clusters_12.pdf
Resolution: 10, Number of clusters: 78
Plot saved to clustering_plots/UMAP_projection_res_10_clusters_78.pdf
